In [1]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
#CONFIGURAÇÕES SPARK
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    .config("spark.executor.memory", "16g")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/09/30 09:26:26 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/30 09:26:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ee29d31-b53c-45c7-afab-2ae8719d53cb;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 158ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
#CONFIGURAÇÕES CLIENT DO MINIO
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [14]:
bucket_name = 'bronze'
objects=client.list_objects(bucket_name, prefix='CAPES/', recursive=True)
csvs = []
for object in objects:
    csvs.append(object.object_name)
    
name_csvs = []
for csv in csvs:
    name_csvs.append(csv.split('/'))

In [15]:
#LISTANDO OS DATAFRAMES PRESENTES NO BUCKET RAW DO MinIO
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv
1 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv
2 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv
3 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv
4 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv
5 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnic

In [16]:
def unionDf(dfstart, dfend):
    df_one = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[dfstart]}')
    for i in range(dfstart+1, dfend+1):
        df_two = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[dfend]}')
        df_one = df_one.unionByName(df_two, allowMissingColumns=True)
    return df_one


In [17]:
def clearOnlyParquets(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.endswith(".parquet"):
                os.remove(os.path.join(root, file))

In [18]:
def toParquet(df, nameDf):
    df.write.parquet(nameDf)
    clearOnlyParquets(nameDf)

In [19]:
programas = unionDf(47,48)

In [20]:
programas = programas.drop('AN_BASE',
                           'ID_ADD_FOTO_PROGRAMA_IES',
                           'ID_ADD_FOTO_PROGRAMA',
                           'IN_REDE'
                          )

['AN_BASE',
 'NM_GRANDE_AREA_CONHECIMENTO',
 'NM_AREA_CONHECIMENTO',
 'NM_AREA_BASICA',
 'NM_SUBAREA_CONHECIMENTO',
 'NM_ESPECIALIDADE',
 'CD_AREA_AVALIACAO',
 'NM_AREA_AVALIACAO',
 'CD_ENTIDADE_CAPES',
 'CD_ENTIDADE_EMEC',
 'SG_ENTIDADE_ENSINO',
 'NM_ENTIDADE_ENSINO',
 'IN_REDE',
 'SG_ENTIDADE_ENSINO_REDE',
 'CS_STATUS_JURIDICO',
 'DS_DEPENDENCIA_ADMINISTRATIVA',
 'DS_ORGANIZACAO_ACADEMICA',
 'NM_REGIAO',
 'SG_UF_PROGRAMA',
 'NM_MUNICIPIO_PROGRAMA_IES',
 'NM_MODALIDADE_PROGRAMA',
 'CD_PROGRAMA_IES',
 'NM_PROGRAMA_IES',
 'NM_PROGRAMA_IDIOMA',
 'NM_GRAU_PROGRAMA',
 'CD_CONCEITO_PROGRAMA',
 'AN_INICIO_PROGRAMA',
 'AN_INICIO_CURSO',
 'DS_SITUACAO_PROGRAMA',
 'DT_SITUACAO_PROGRAMA',
 'ID_ADD_FOTO_PROGRAMA_IES',
 'ID_ADD_FOTO_PROGRAMA']